# 根据神经网络，对用户排序

-   用户集
-   顺序无关神经网络（神经网络类型和state_dict）
-   排序后的存储

In [1]:
WORKSPACEFOLDER = 'E:\Projects\ComplexNetwork'
import os
os.chdir(WORKSPACEFOLDER)

In [10]:
import json
import functools

import torch

In [3]:
import config

DB = config.get_DB()

In [4]:
import matplotlib.pyplot as plt

%matplotlib notebook

In [7]:
import src.main.util.sort as sort

## SNAPTwitter

### 条件： cn20 nodes20_beta0.0120

In [5]:
with open(r'db\SNAPTwitter\cn 20.json', 'r') as fr:
    cn20 = json.load(fr)

In [12]:
folder_nn = 'db/SNAPTwitter/nn/order_independent_net'
file_emb = 'db/SNAPTwitter/SNAPTwitter.node2vec.emb.json'
cmp = sort.sort_by_cmp(folder_nn, file_emb)

In [13]:
cn20_sorted = []
for i in range(len(cn20)):
    cn20_sorted.append(sorted(cn20[i], key=functools.cmp_to_key(cmp), reverse=True))

In [14]:
cn20_sorted[0]

[70106,
 39025,
 50461,
 49112,
 61473,
 48714,
 28509,
 41545,
 40903,
 62757,
 696,
 63724,
 78318,
 79784,
 23673,
 51277,
 570,
 44879,
 73349,
 1401,
 45464,
 63363,
 23247,
 71668,
 37237,
 68800,
 4765,
 63745,
 29754,
 66200,
 30844,
 44825,
 12094,
 7312,
 2676,
 80753,
 80581,
 72642,
 32019,
 56110,
 11523,
 55752,
 41786,
 4397,
 13899,
 57062,
 36479,
 9241,
 79490,
 27770,
 13165,
 52889,
 29862,
 13031,
 51588,
 47943,
 58663,
 1529,
 31260,
 25757,
 12316,
 23973,
 40035,
 77191,
 16285,
 73964,
 10457,
 4691,
 24972,
 36289,
 71689,
 66320,
 71831,
 54194,
 31432,
 58205,
 48279,
 7060,
 69765,
 40232,
 1706,
 45692,
 30850,
 77599,
 69928,
 19275,
 52879,
 17916,
 12785,
 75654,
 49977,
 54799,
 32622,
 4704,
 25447,
 35986,
 18037,
 76185,
 10364,
 28325]